In [1]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import scipy.stats as stats

plt.rcParams.update({'font.size': 14})
pd.set_option('display.float_format', lambda x: '%.2f' % x)
path = '../../../../playwright/results/core-web-vitals/testrun-8/'


In [2]:
df = pd.read_csv(f'{path}2024-12-19T09:12:33.629Z_results-monolith.csv', sep=',')
df_t = pd.read_csv(f'{path}2024-12-19T10:13:15.254Z_results-monolith-throttled.csv', sep=',')

features = ['navTime', 'totalTime', 'lcp', 'fcp', 'ttfb', 'tbt', 'tti', 'longestTask', 'longTasks', 'nf:init', 'nf:config','nf:loaded']

dirty_dfs = {
    'unthrottled': df[5:],
    'throttled': df_t[5:],
  #  'heavy': df_ht[5:],
}

In [3]:
def detect_outliers(_df, _features, contamination=0.1):
    clf = IsolationForest(contamination=contamination, random_state=42)
    outliers = clf.fit_predict(_df[_features])
    return outliers == 1

masks = {}
dfs = {}
target_features = ['navTime', 'totalTime', 'lcp', 'fcp', 'ttfb']

for name, _df in dirty_dfs.items():
    mask = detect_outliers(_df, target_features)
    masks[name] = mask
    dfs[name] = _df[mask].copy()

In [4]:
columns = [ 'ttfb','fcp','nf:init','lcp','tti','nf:loaded','tbt','longestTask']
rows = []

for name, df in dfs.items():
    mean_row = df[columns].mean()
    percentile_row = df[columns].quantile(0.75)
    
    rows.append((f"{name} (mean)", mean_row))
    rows.append((f"{name} (75th)", percentile_row))

result_df = pd.DataFrame([row[1] for row in rows], index=[row[0] for row in rows])
result_df = result_df.mask(result_df < 0, '-')
result_df

,ttfb,fcp,nf:init,lcp,tti,nf:loaded,tbt,longestTask
unthrottled (mean),3.74,70.91,-,70.91,70.91,-,0.00,-
unthrottled (75th),4.00,72.40,-,72.40,72.40,-,0.00,-
throttled (mean),354.21,1144.63,-,1144.63,1144.63,-,0.00,-
throttled (75th),354.80,1151.50,-,1151.50,1151.50,-,0.00,-
